In [89]:
from __future__ import unicode_literals, division
import json
from collections import Counter, defaultdict
import pandas as pd

In [90]:
base_ht_df = pd.read_csv('../../data/expanded_hashtags.csv', encoding='UTF8')
exp_ht_df = pd.read_csv('../../data/expanded_fixed_hashtags.csv', encoding='UTF8')

In [91]:
all_ht_df = base_ht_df.merge(exp_ht_df, how='outer', left_on='hashtag', right_on='original')
pro_tags = [h.strip() for h in all_ht_df[all_ht_df.sentiment == 'pro']['expanded']]
anti_tags = [h.strip() for h in all_ht_df[all_ht_df.sentiment == 'anti']['expanded']]
#print pro_tags, anti_tags
print len(pro_tags), len(anti_tags)

38 45


In [92]:
dec = json.JSONDecoder()
user_counts = defaultdict(Counter)
with open('../../data/tweets/archive_Jan-01-17_Oct-31-17_ref_hashtags_filtered.json') as json_file:
    for l in json_file:
        record = dec.decode(l)
        hashtags = [t.strip()[1:] for t in record['text'].split() if t.startswith('#')]
        pro_ct = len([h for h in hashtags if h in pro_tags])
        anti_ct = len([h for h in hashtags if h in anti_tags])
        if pro_ct > 0:
            user_counts[record['user']]['pro'] += 1
        if anti_ct > 0:
            user_counts[record['user']]['anti'] += 1
print len(user_counts)

def pro_rate(user_ct):
    return user_ct['pro'] / sum(user_ct.values())

pro_users = [u for u,c in user_counts.items() if pro_rate(c) >= 0.75]
anti_users = [u for u,c in user_counts.items() if pro_rate(c) <= 0.25]

print len(pro_users), len(anti_users)

1422
1084 327


In [93]:
out_df = pd.concat([pd.DataFrame(data={'username':pro_users, 'group':'YES'}),
                    pd.DataFrame(data={'username':anti_users, 'group':'NO'})])
len(out_df2)

1411

In [94]:
out_df.head()

,group,username
0,YES,TTrendingTopicc
1,YES,zibzeg
2,YES,ddos_mg
3,YES,jeff_puente
4,YES,alvarorickman


In [95]:
out_df.to_csv('../../data/user_groups.csv', index=False, encoding='UTF8')